# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [58]:
# Write your code below.
%load_ext dotenv
%dotenv 


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [59]:
import dask.dataframe as dd
import pandas as pd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [60]:
import os
from glob import glob

# Write your code below.
price_data = os.getenv('PRICE_DATA')
parquet_path = glob(os.path.join(price_data, "**/*.parquet"), recursive=True)
parquet_path


['../../05_src/data/prices\\ACN\\ACN_2001\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2001\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2002\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2002\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2003\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2003\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2004\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2004\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2005\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2005\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2006\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2006\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2007\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2007\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2008\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2008\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_200

For each ticker and using Dask, do the following:
`
+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [63]:
dd_px1 = dd.read_parquet(parquet_path).set_index("ticker")
dd_feat = (
    dd_px1.groupby('ticker', group_keys=False)
    .apply(lambda x: x.sort_values('Date', ascending=True)
           .assign(Close_lag_1=x['Close'].shift(1),
                   Adj_Close_lag_1=x['Adj Close'].shift(1),
                   Returns=x['Close'] / x['Close'].shift(1) - 1,
                   Hi_Lo_Range=x['High'] - x['Low']),
            meta = pd.DataFrame(data ={
                    'Date': 'datetime64[ns]',
                    'Open': 'f8',
                    'High': 'f8',
                    'Low': 'f8',
                    'Close': 'f8',
                    'Adj Close': 'f8',
                    'Volume': 'i8',
                    'source': 'object',
                    'Year': 'int32',
                    'Close_lag_1': 'f8',
                    'Adj_Close_lag_1':'f8',
                    'Returns':'f8',
                    'Hi_Lo_Range':'f8'
                    },
                    index = pd.Index([], dtype=pd.StringDtype(), name='ticker'))  
                   )
)
dd_feat

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,Returns,Hi_Lo_Range
npartitions=89,,,,,,,,,,,,,
ACN,object,object,object,object,object,object,object,object,object,object,object,object,object
AFGC,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZIXI,...,...,...,...,...,...,...,...,...,...,...,...,...
ZIXI,...,...,...,...,...,...,...,...,...,...,...,...,...


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [62]:
# Write your code below.
dd_panda = dd_feat.compute()
dd_panda['Moving_Average_10_Days'] = dd_panda.groupby('ticker')['Returns'].transform(lambda x: x.rolling(10).mean())
dd_panda


,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,Returns,Hi_Lo_Range,Moving_Average_10_Days
ticker,,,,,,,,,,,,,,
ACN,2001-07-19,15.10,15.29,15.00,15.17,11.404394,34994300.0,ACN.csv,2001,NaN,NaN,NaN,0.290000,NaN
ACN,2001-07-20,15.05,15.05,14.80,15.01,11.284108,9238500.0,ACN.csv,2001,15.17,11.404394,-0.010547,0.250000,NaN
ACN,2001-07-23,15.00,15.01,14.55,15.00,11.276587,7501000.0,ACN.csv,2001,15.01,11.284108,-0.000666,0.460000,NaN
ACN,2001-07-24,14.95,14.97,14.70,14.86,11.171341,3537300.0,ACN.csv,2001,15.00,11.276587,-0.009333,0.270000,NaN
ACN,2001-07-25,14.70,14.95,14.65,14.95,11.238999,4208100.0,ACN.csv,2001,14.86,11.171341,0.006057,0.300000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZIXI,2020-03-26,4.06,4.53,3.88,4.51,4.510000,1668500.0,ZIXI.csv,2020,7.62,7.620000,0.015748,0.460000,0.014216
ZIXI,2020-03-27,4.49,4.71,4.10,4.60,4.600000,1146800.0,ZIXI.csv,2020,7.74,7.740000,0.041344,0.330000,0.018939
ZIXI,2020-03-30,4.83,4.87,4.44,4.64,4.640000,1212000.0,ZIXI.csv,2020,8.06,8.060000,-0.022333,0.290000,0.010632


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

No, it was not necessary to to convert to pandas to calculate the moving average return. Dask could have performed the same function. 

It would have been better to perform the calculation in Dask because the dataset was large. Using Pandas required a large amount of memory for this large dataset, thus my computer performance was slow. Dask is quicker for performance with large datasets.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.